In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from PIL import Image
import numpy as np
#numpy.array(Image.fromarray(arr).resize())

In [ ]:
pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 143kB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
!pip3 install scikit-image
from skimage.transform import resize

In [ ]:
import imageio


In [ ]:
import scipy.misc
import random

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0
tk = 0
#read data.txt
with open("/content/drive/MyDrive/Case-2: Self Driving Car/driving_dataset/data.txt") as f:
    for line in f:
        xs.append("/content/drive/MyDrive/Case-2: Self Driving Car/driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)
        tk = tk +1
        if tk ==5000 :
            break

#get number of images
num_images = len(xs)
print(num_images)

train_xs = xs[:int(len(xs) * 0.7)]
train_ys = ys[:int(len(xs) * 0.7)]
print(len(train_xs))
val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]
print(len(val_xs))
num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(resize(imageio.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66, 200]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(resize(imageio.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out


5000
3500
1500


#Model

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import scipy

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32)
y_= tf.placeholder(tf.float32)


x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.multiply(tf.atan(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) #scale the atan output


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
train_vars = tf.trainable_variables()

In [ ]:
for j in train_vars:
    print(j)

<tf.Variable 'Variable:0' shape=(5, 5, 3, 24) dtype=float32_ref>
<tf.Variable 'Variable_1:0' shape=(24,) dtype=float32_ref>
<tf.Variable 'Variable_2:0' shape=(5, 5, 24, 36) dtype=float32_ref>
<tf.Variable 'Variable_3:0' shape=(36,) dtype=float32_ref>
<tf.Variable 'Variable_4:0' shape=(5, 5, 36, 48) dtype=float32_ref>
<tf.Variable 'Variable_5:0' shape=(48,) dtype=float32_ref>
<tf.Variable 'Variable_6:0' shape=(3, 3, 48, 64) dtype=float32_ref>
<tf.Variable 'Variable_7:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'Variable_8:0' shape=(3, 3, 64, 64) dtype=float32_ref>
<tf.Variable 'Variable_9:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'Variable_10:0' shape=(1152, 1164) dtype=float32_ref>
<tf.Variable 'Variable_11:0' shape=(1164,) dtype=float32_ref>
<tf.Variable 'Variable_12:0' shape=(1164, 100) dtype=float32_ref>
<tf.Variable 'Variable_13:0' shape=(100,) dtype=float32_ref>
<tf.Variable 'Variable_14:0' shape=(100, 50) dtype=float32_ref>
<tf.Variable 'Variable_15:0' shape=(50,) dtype=f

In [ ]:
import time
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.core.protobuf import saver_pb2


LOGDIR = ("/content/drive/MyDrive/Case-2: Self Driving Car/save")

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_,y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
#merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

# op to write logs to Tensorboard
#logs_path = r'/content/drive/MyDrive/Case-2: Self Driving Car/logss'
#summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 3
batch_size = 10

# train over the dataset about 30 times
for epoch in range(epochs):
  #start = time.time()
  #localtime = time.asctime( time.localtime(time.time()) )
  #print ("Local current time :", localtime)

  for i in range(int(num_images/batch_size)):
    xs, ys = LoadTrainBatch(batch_size)
    train_step.run(feed_dict={x: xs, y_: ys,keep_prob: 0.8})
    xs, ys = LoadValBatch(batch_size)
    loss_value = loss.eval(feed_dict={x:xs,y_: ys,keep_prob: 1.0})
    print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))
    #end = time.time()
    #print('Time took to Execuate: ',end-start,'sec')
    #start = time.time()

    # write logs at every iteration
    #summary = merged_summary_op.eval(feed_dict={x:xs,y_: ys,keep_prob: 1.0})
    #summary_writer.add_summary(summary, epoch * num_images/batch_size + i)


  checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
  filename = saver.save(sess, checkpoint_path)
  #localtime = time.asctime( time.localtime(time.time()) )
  #print ("Local current time :", localtime)

#print("Run the command line:\n" \
          #"--> tensorboard --logdir=./logs " \
          #"\nThen open http://0.0.0.0:6006/ into your web browser")


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0, Step: 0, Loss: 6.20063
Epoch: 0, Step: 1, Loss: 6.17123
Epoch: 0, Step: 2, Loss: 6.15658
Epoch: 0, Step: 3, Loss: 6.1483
Epoch: 0, Step: 4, Loss: 6.13944
Epoch: 0, Step: 5, Loss: 6.12999
Epoch: 0, Step: 6, Loss: 6.11927
Epoch: 0, Step: 7, Loss: 6.10893
Epoch: 0, Step: 8, Loss: 6.09867
Epoch: 0, Step: 9, Loss: 6.0898
Epoch: 0, Step: 10, Loss: 6.08011
Epoch: 0, Step: 11, Loss: 6.07063
Epoch: 0, Step: 12, Loss: 6.06349
Epoch: 0, Step: 13, Loss: 6.05787
Epoch: 0, Step: 14, Loss: 6.04744
Epoch: 0, Step: 15, Loss: 6.0376
Epoch: 0, Step: 16, Loss: 6.02915
Epoch: 0, Step: 17, Loss: 6.019
Epoch: 0, Step: 18, Loss: 6.01011
Epoch: 0, Step: 19, Loss: 6.0002
Epoch: 0, Step: 20, Loss: 5.99133
Epoch: 0, Step: 21, Loss: 5.98251
Epoch: 0, Step: 22, Loss: 5.97382
Epoch: 0, Step: 23, Loss: 5.96458
Epoch: 0, Step: 24, Loss: 5.95571
Epoch: 0, Step: 25, Loss: 5.94837
Epoch: 0, Step: 26, Loss: 5.94091
Epoch: 0, Step: 27, Loss

KeyboardInterrupt: ignored